In [1]:
# Импортируем библиотеки

import numpy as np
import pandas as pd
import seaborn as sns
sns.set_palette('husl')
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn.linear_model import LassoCV

In [2]:
data = pd.read_csv('heart.csv')
data.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [4]:
# Преобразование категориальных данных в количественные
data['Sex'] = data['Sex'].map({'M': 1, 'F': 0})
data['ChestPainType'] = data['ChestPainType'].map({'ATA': 0, 'NAP': 1, 'ASY': 2, 'TA': 3})
data['RestingECG'] = data['RestingECG'].map({'Normal': 0, 'ST': 1, 'LVH': 2})
data['ExerciseAngina'] = data['ExerciseAngina'].map({'N': 0, 'Y': 1})
data['ST_Slope'] = data['ST_Slope'].map({'Up': 0, 'Flat': 1, 'Down': 2})

In [5]:
data.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,1,0,140,289,0,0,172,0,0.0,0,0
1,49,0,1,160,180,0,0,156,0,1.0,1,1
2,37,1,0,130,283,0,1,98,0,0.0,0,0
3,48,0,2,138,214,0,0,108,1,1.5,1,1
4,54,1,1,150,195,0,0,122,0,0.0,0,0


In [6]:
X = data.drop(['HeartDisease'], axis=1)
y = data['HeartDisease']
print(f'X shape: {X.shape} | y shape: {y.shape} ')

X shape: (918, 11) | y shape: (918,) 


In [7]:
import scipy.stats as stats

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, shuffle=False)

stats.ttest_ind (a=y_train, b=y_test)

TtestResult(statistic=np.float64(3.4545395326502772), pvalue=np.float64(0.0005765173438610094), df=np.float64(916.0))

In [29]:
knn = KNeighborsClassifier()
grid_param = {'n_neighbors':range(1,30)}
grid_model = GridSearchCV(estimator = knn, param_grid = grid_param, scoring = 'accuracy', cv = 5, verbose = 0)
grid_model.fit(X_train, y_train)
grid_optimal=grid_model.best_params_
print(grid_optimal)

{'n_neighbors': 28}


In [30]:
predictions = grid_model.predict(X_test)

# Вычисление коэффициент детерминации
from sklearn.metrics import r2_score
r2 = r2_score(y_test, predictions)
print('r2=', r2)

# Вычисление среднекваратичной ошибки (MSE и RMSE)
from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(y_test, predictions)
print('MSE=', MSE)

from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, predictions))
print('RMSE=', RMSE)

r2= -0.4263565891472867
MSE= 0.35507246376811596
RMSE= 0.595879571531124


In [10]:
parameters = {'kernel':('linear', 'rbf','poly','sigmoid','precomputer'), 'C':[i for i in range(1,7)]}

svc = SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(X_train, y_train)
print("Наилучшие значения параметров: ", clf.best_params_)

C:\Users\кря\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
30 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\кря\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\кря\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "C:\Users\кря\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py", line

Наилучшие значения параметров:  {'C': 5, 'kernel': 'linear'}


In [28]:
predictions = clf.predict(X_test)

# Вычисление коэффициент детерминации
from sklearn.metrics import r2_score
r2 = r2_score(y_test, predictions)
print('r2=', r2)

# Вычисление среднекваратичной ошибки (MSE и RMSE)
from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(y_test, predictions)
print('MSE=', MSE)

from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, predictions))
print('RMSE=', RMSE)

r2= -0.10615408954279393
MSE= 0.2753623188405797
RMSE= 0.5247497678328021


In [12]:
tree = DecisionTreeClassifier()

In [39]:
tree.fit(X_train, y_train)
predictions = tree.predict(X_test)

In [40]:
# Вычисление коэффициент детерминации
from sklearn.metrics import r2_score
r2 = r2_score(y_test, predictions)
print('r2=', r2)

# Вычисление среднекваратичной ошибки (MSE и RMSE)
from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(y_test, predictions)
print('MSE=', MSE)

from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, predictions))
print('RMSE=', RMSE)

r2= -0.30991931656383476
MSE= 0.32608695652173914
RMSE= 0.5710402407201608
